In [49]:
from attributes import *
import polars as pl
import altair as alt
from pathlib import Path

Path("figure/3/").mkdir(parents=True, exist_ok=True)

df = pl.read_csv(
    "data/3_research_ethics_education.csv",
)

print(df)

shape: (167, 65)
┌───────┬─────┬──────┬──────┬───┬────────────────────┬─────┬──────┬────────────────────────────────┐
│ ID    ┆ Q6  ┆ Q7_1 ┆ Q7_2 ┆ … ┆ Q17_5_duplicated_0 ┆ Q18 ┆ Q19  ┆ Q20                            │
│ ---   ┆ --- ┆ ---  ┆ ---  ┆   ┆ ---                ┆ --- ┆ ---  ┆ ---                            │
│ i64   ┆ i64 ┆ i64  ┆ i64  ┆   ┆ str                ┆ i64 ┆ str  ┆ str                            │
╞═══════╪═════╪══════╪══════╪═══╪════════════════════╪═════╪══════╪════════════════════════════════╡
│ 14630 ┆ 1   ┆ null ┆ null ┆ … ┆ null               ┆ 2   ┆ null ┆ .                              │
│ 14629 ┆ 1   ┆ null ┆ null ┆ … ┆ null               ┆ 2   ┆ null ┆ null                           │
│ 14626 ┆ 1   ┆ 1    ┆ null ┆ … ┆ null               ┆ 2   ┆ null ┆ null                           │
│ 14624 ┆ 2   ┆ null ┆ null ┆ … ┆ null               ┆ 2   ┆ null ┆ null                           │
│ 14627 ┆ 1   ┆ null ┆ null ┆ … ┆ null               ┆ 2   ┆ null ┆ null  

In [50]:
def count_y(name):
    return df.group_by(pl.col(name).alias("Y")).len().rename({"len":name})   

def count_list_y(columns, select):
    df = pl.DataFrame({"Y":select})
    for col in columns:
        count = df.group_by(pl.col(col).alias("Y")).len().rename({"len":col})
        df = df.join(count, on="Y", how="left")
    return df

def draw_bar_plot(df, x, y, title, MAP, width=400):

    map = MAP
    if type(MAP) is dict:
        map = MAP.values()

    bar_x = alt.X(x, title=None, axis=None)
    bar_x = bar_x if MAP is None else bar_x.sort(map)

    txt_x = alt.X(x, sort=map) if MAP is not None else alt.X(x)

    bar = df.plot.bar().encode(
        x=bar_x,
        y=alt.Y(y, title=None),
        color=alt.Color(x, title=None, scale=alt.Scale(scheme="tableau20")).sort(map),
    ).properties(
        title=title,
        width=width,
    )

    text = bar.mark_text(
        align="center",
        baseline="bottom",
    ).encode(
        x=txt_x,
        y=y,
        text=y,
        color=alt.value("black"),
    )
    return bar + text

In [51]:
def basic_analysis(name):
    count = df.group_by(name).len().sort(name).drop_nulls()
    proportion = count.with_columns(
        pct = (pl.col("len") / pl.col("len").sum() * 100).round(1)
    )
    return count, proportion, proportion.sum()

In [52]:
from attributes import *
from scipy import stats
import pandas as pd

#
# Chi-square test
#
personal_info = pl.read_csv("data/6_personal_info.csv")
recognition_stats = df.join(personal_info, on="ID")

def chi2_test(target_name):
    chi_analysis = {}
    for key, value in MAP_PERSON.items():
        target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)
        t, p, dof, _ = stats.chi2_contingency(target.to_numpy())
        chi_analysis[key] = (float(t), float(p))
    return chi_analysis

In [53]:
# 
# 가 연구자의 연구윤리 교육 경험
# 

# 연구윤리 규범 범위의 확대 인식 여부
columns = {
    "Q6": "연구윤리 규범 범위의 확대 인식 여부",
}

research_ethics_eduction_experience = df.group_by("Q6").len()
research_ethics_eduction_experience.with_columns(
    (pl.col("Q6") / pl.col("Q6").sum()).alias("percentage"),
)

Q6,len,percentage
i64,u32,f64
2,30,0.666667
1,137,0.333333


In [54]:
# 
# 나. 연구윤리 교육 방법
# 

map = ["원내 집합 교육", "원외 집합 교육", "원내 온라인 강의", "원외 온라인 강의", "기타"]

method = df.select([
    pl.col("Q7_1").value_counts().sort().struct[1].alias(map[0]),
    pl.col("Q7_2").value_counts().sort().struct[1].alias(map[1]),
    pl.col("Q7_3").value_counts().sort().struct[1].alias(map[2]),
    pl.col("Q7_4").value_counts().sort().struct[1].alias(map[3]),
    pl.col("Q7_5").value_counts().sort().struct[1].alias(map[4]),
])[1]

method.write_csv("figure/3/나_연구윤리_교육_방법.csv")
pct = method.with_columns(
    pl.col(map[0]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4])).alias(map[0]),
    pl.col(map[1]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4])).alias(map[1]),
    pl.col(map[2]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4])).alias(map[2]),
    pl.col(map[3]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4])).alias(map[3]),
    pl.col(map[4]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4])).alias(map[4]),
)
print(method)
print(pct)


shape: (1, 5)
┌────────────────┬────────────────┬──────────────────┬──────────────────┬──────┐
│ 원내 집합 교육 ┆ 원외 집합 교육 ┆ 원내 온라인 강의 ┆ 원외 온라인 강의 ┆ 기타 │
│ ---            ┆ ---            ┆ ---              ┆ ---              ┆ ---  │
│ u32            ┆ u32            ┆ u32              ┆ u32              ┆ u32  │
╞════════════════╪════════════════╪══════════════════╪══════════════════╪══════╡
│ 30             ┆ 4              ┆ 113              ┆ 26               ┆ 2    │
└────────────────┴────────────────┴──────────────────┴──────────────────┴──────┘
shape: (1, 5)
┌────────────────┬────────────────┬──────────────────┬──────────────────┬──────────┐
│ 원내 집합 교육 ┆ 원외 집합 교육 ┆ 원내 온라인 강의 ┆ 원외 온라인 강의 ┆ 기타     │
│ ---            ┆ ---            ┆ ---              ┆ ---              ┆ ---      │
│ f64            ┆ f64            ┆ f64              ┆ f64              ┆ f64      │
╞════════════════╪════════════════╪══════════════════╪══════════════════╪══════════╡
│ 0.171429       ┆ 0.022857       ┆ 

In [55]:
# 
# 다. 연구윤리 교육 내용
# 

map = ["책임 있는 연구수행", "출판윤리", "연구 공동체", "연구자의 사회적 책임", "연구 성과 및 데이터", "이해충돌", "생명윤리", "연구진실성 검증", "학문교류 윤리", "부실학술활동 예방", "ChatGPT 등 생성형 AI 활용과 연구윤리"]

contents = df.select([
    pl.col("Q8_1").value_counts().sort().struct[1].alias(map[0]),
    pl.col("Q8_2").value_counts().sort().struct[1].alias(map[1]),
    pl.col("Q8_3").value_counts().sort().struct[1].alias(map[2]),
    pl.col("Q8_4").value_counts().sort().struct[1].alias(map[3]),
    pl.col("Q8_5").value_counts().sort().struct[1].alias(map[4]),
    pl.col("Q8_6").value_counts().sort().struct[1].alias(map[5]),
    pl.col("Q8_7").value_counts().sort().struct[1].alias(map[6]),
    pl.col("Q8_8").value_counts().sort().struct[1].alias(map[7]),
    pl.col("Q8_9").value_counts().sort().struct[1].alias(map[8]),
    pl.col("Q8_10").value_counts().sort().struct[1].alias(map[9]),
    pl.col("Q8_11").value_counts().sort().struct[1].alias(map[10]),
])[1]

contents.write_csv("figure/3/다_연구윤리_교육_내용.csv")
print(contents)

pct = contents.with_columns(
    pl.col(map[0]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4]) + pl.col(map[5]) + pl.col(map[6]) + pl.col(map[7]) + pl.col(map[8]) + pl.col(map[9]) + pl.col(map[10])).alias(map[0]),
    pl.col(map[1]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4]) + pl.col(map[5]) + pl.col(map[6]) + pl.col(map[7]) + pl.col(map[8]) + pl.col(map[9]) + pl.col(map[10])).alias(map[1]),
    pl.col(map[2]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4]) + pl.col(map[5]) + pl.col(map[6]) + pl.col(map[7]) + pl.col(map[8]) + pl.col(map[9]) + pl.col(map[10])).alias(map[2]),
    pl.col(map[3]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4]) + pl.col(map[5]) + pl.col(map[6]) + pl.col(map[7]) + pl.col(map[8]) + pl.col(map[9]) + pl.col(map[10])).alias(map[3]),
    pl.col(map[4]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4]) + pl.col(map[5]) + pl.col(map[6]) + pl.col(map[7]) + pl.col(map[8]) + pl.col(map[9]) + pl.col(map[10])).alias(map[4]),
    pl.col(map[5]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4]) + pl.col(map[5]) + pl.col(map[6]) + pl.col(map[7]) + pl.col(map[8]) + pl.col(map[9]) + pl.col(map[10])).alias(map[5]),
    pl.col(map[6]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4]) + pl.col(map[5]) + pl.col(map[6]) + pl.col(map[7]) + pl.col(map[8]) + pl.col(map[9]) + pl.col(map[10])).alias(map[6]),
    pl.col(map[7]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4]) + pl.col(map[5]) + pl.col(map[6]) + pl.col(map[7]) + pl.col(map[8]) + pl.col(map[9]) + pl.col(map[10])).alias(map[7]),
    pl.col(map[8]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4]) + pl.col(map[5]) + pl.col(map[6]) + pl.col(map[7]) + pl.col(map[8]) + pl.col(map[9]) + pl.col(map[10])).alias(map[8]),
    pl.col(map[9]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4]) + pl.col(map[5]) + pl.col(map[6]) + pl.col(map[7]) + pl.col(map[8]) + pl.col(map[9]) + pl.col(map[10])).alias(map[9]),
    pl.col(map[10]) / (pl.col(map[0]) + pl.col(map[1]) + pl.col(map[2]) + pl.col(map[3]) + pl.col(map[4]) + pl.col(map[5]) + pl.col(map[6]) + pl.col(map[7]) + pl.col(map[8]) + pl.col(map[9]) + pl.col(map[10])).alias(map[10]),
)
print(pct)

bar = draw_bar_plot(contents.transpose(include_header=True), "column", "column_0", "연구윤리 교육 내용", map, width=800)
bar.save("figure/3/다_연구윤리_교육_내용.png")

shape: (1, 11)
┌────────────┬──────────┬────────┬────────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ 책임 있는  ┆ 출판윤리 ┆ 연구   ┆ 연구자의   ┆ … ┆ 연구진실성 ┆ 학문교류  ┆ 부실학술  ┆ ChatGPT   │
│ 연구수행   ┆ ---      ┆ 공동체 ┆ 사회적     ┆   ┆ 검증       ┆ 윤리      ┆ 활동 예방 ┆ 등 생성형 │
│ ---        ┆ u32      ┆ ---    ┆ 책임       ┆   ┆ ---        ┆ ---       ┆ ---       ┆ AI 활용과 │
│ u32        ┆          ┆ u32    ┆ ---        ┆   ┆ u32        ┆ u32       ┆ u32       ┆ 연구윤리  │
│            ┆          ┆        ┆ u32        ┆   ┆            ┆           ┆           ┆ ---       │
│            ┆          ┆        ┆            ┆   ┆            ┆           ┆           ┆ u32       │
╞════════════╪══════════╪════════╪════════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ 93         ┆ 93       ┆ 30     ┆ 34         ┆ … ┆ 41         ┆ 34        ┆ 49        ┆ 18        │
└────────────┴──────────┴────────┴────────────┴───┴────────────┴───────────┴───────────┴───────────┘
shape: (1, 11)
┌───────────┬─────

In [56]:
# 
# 라. 연구윤리 교육 만족도
# 
# 1)
column = {
    "Q9": "연구윤리 교육 만족도",
}

count = df.group_by("Q9").len().drop_nulls().sort("Q9")
all = count.with_columns((pl.col("len") / pl.col("len").sum()).alias("percentage")).rename(column)
print(all)
all.write_csv("figure/3/라_1_연구윤리_교육_만족도.csv", include_bom=True)

shape: (4, 3)
┌──────────────────────┬─────┬────────────┐
│ 연구윤리 교육 만족도 ┆ len ┆ percentage │
│ ---                  ┆ --- ┆ ---        │
│ i64                  ┆ u32 ┆ f64        │
╞══════════════════════╪═════╪════════════╡
│ 2                    ┆ 3   ┆ 0.021898   │
│ 3                    ┆ 35  ┆ 0.255474   │
│ 4                    ┆ 80  ┆ 0.583942   │
│ 5                    ┆ 19  ┆ 0.138686   │
└──────────────────────┴─────┴────────────┘


In [57]:
# 
# 라. 연구윤리 교육 만족도
# 
# 2)
column = {
    "Q10": "연구윤리 교육 만족의 이유",
}

count = df.group_by("Q10").len().drop_nulls().sort("Q10")
all = count.with_columns((pl.col("len") / pl.col("len").sum()).alias("percentage")).rename(column)
print(all)
all.write_csv("figure/3/라_2_연구윤리_교육_만족_이유.csv", include_bom=True)

shape: (3, 3)
┌───────────────────────────┬─────┬────────────┐
│ 연구윤리 교육 만족의 이유 ┆ len ┆ percentage │
│ ---                       ┆ --- ┆ ---        │
│ i64                       ┆ u32 ┆ f64        │
╞═══════════════════════════╪═════╪════════════╡
│ 1                         ┆ 22  ┆ 0.222222   │
│ 2                         ┆ 9   ┆ 0.090909   │
│ 3                         ┆ 68  ┆ 0.686869   │
└───────────────────────────┴─────┴────────────┘


In [58]:
# 
# 라. 연구윤리 교육 만족도
# 
# 3)
column = {
    "Q11": "연구윤리 교육 불만족의 이유",
}

count = df.group_by("Q11").len().drop_nulls().sort("Q11")
all = count.with_columns((pl.col("len") / pl.col("len").sum()).alias("percentage")).rename(column)
all.write_csv("figure/3/라_2_연구윤리_교육_불만족_이유.csv", include_bom=True)

In [59]:
# 
# 마. 연구자가 향후 받고 싶은 교육 내용
# 

map = ["연구부정행위 사례 및 검증", "출판윤리", "연구비 집행", "연구노트 작성 및 데이터 관리", "연구자의 사회적 책임", "부실 학술활동 예방", "AI 활용 연구활동과 연구윤리 누락", "이해충돌 관리", "기타"]

future_contents = df.select([
    pl.col("Q12_1").value_counts().sort().struct[1].alias(map[0]),
    pl.col("Q12_2").value_counts().sort().struct[1].alias(map[1]),
    pl.col("Q12_3").value_counts().sort().struct[1].alias(map[2]),
    pl.col("Q12_4").value_counts().sort().struct[1].alias(map[3]),
    pl.col("Q12_5").value_counts().sort().struct[1].alias(map[4]),
    pl.col("Q12_6").value_counts().sort().struct[1].alias(map[5]),
    pl.col("Q12_7").value_counts().sort().struct[1].alias(map[6]),
    pl.col("Q12_8").value_counts().sort().struct[1].alias(map[7]),
    pl.col("Q12_9").value_counts().sort().struct[1].alias(map[8]),
])[1]

print(future_contents)
future_contents.write_csv("figure/3/마_연구자가_향후_받고_싶은_교육_내용.csv", include_bom=True)

bar = draw_bar_plot(future_contents.transpose(include_header=True), "column", "column_0", "연구자가 향후 받고 싶은 교육 내용", map)
bar.save("figure/3/마_연구자가_향후_받고_싶은_교육_내용.svg")

shape: (1, 9)
┌─────────────┬──────────┬────────┬─────────────┬───┬─────────────┬────────────┬────────────┬──────┐
│ 연구부정행  ┆ 출판윤리 ┆ 연구비 ┆ 연구노트    ┆ … ┆ 부실        ┆ AI 활용    ┆ 이해충돌   ┆ 기타 │
│ 위 사례 및  ┆ ---      ┆ 집행   ┆ 작성 및     ┆   ┆ 학술활동    ┆ 연구활동과 ┆ 관리       ┆ ---  │
│ 검증        ┆ u32      ┆ ---    ┆ 데이터 관리 ┆   ┆ 예방        ┆ 연구윤리   ┆ ---        ┆ u32  │
│ ---         ┆          ┆ u32    ┆ ---         ┆   ┆ ---         ┆ 누락       ┆ u32        ┆      │
│ u32         ┆          ┆        ┆ u32         ┆   ┆ u32         ┆ ---        ┆            ┆      │
│             ┆          ┆        ┆             ┆   ┆             ┆ u32        ┆            ┆      │
╞═════════════╪══════════╪════════╪═════════════╪═══╪═════════════╪════════════╪════════════╪══════╡
│ 93          ┆ 73       ┆ 75     ┆ 35          ┆ … ┆ 35          ┆ 93         ┆ 33         ┆ 1    │
└─────────────┴──────────┴────────┴─────────────┴───┴─────────────┴────────────┴────────────┴──────┘


In [60]:
# 
# 바. 연구자가 향후 받고 싶은 교육 방법
# 

map = ["대면", "비대면", "사례중심", "토의/토론 중심", "강의 중심", "동영상 등 ICT 활용", "기타"]

future_method = df.select([
    pl.col("Q13_1").value_counts().sort().struct[1].alias(map[0]),
    pl.col("Q13_2").value_counts().sort().struct[1].alias(map[1]),
    pl.col("Q13_3").value_counts().sort().struct[1].alias(map[2]),
    pl.col("Q13_4").value_counts().sort().struct[1].alias(map[3]),
    pl.col("Q13_5").value_counts().sort().struct[1].alias(map[4]),
    pl.col("Q13_6").value_counts().sort().struct[1].alias(map[5]),
    # pl.col("Q13_7").value_counts().sort().struct[1].alias(map[6]),
])[1]

print(future_method)

future_method.write_csv("figure/3/바_연구자가_향후_받고_싶은_교육_방법.csv", include_bom=True)

bar = draw_bar_plot(future_method.transpose(include_header=True), "column", "column_0", "연구자가 향후 받고 싶은 교육 방법", map)
bar.save("figure/3/바_연구자가_향후_받고_싶은_교육_방법.svg")

shape: (1, 6)
┌──────┬────────┬──────────┬────────────────┬───────────┬────────────────────┐
│ 대면 ┆ 비대면 ┆ 사례중심 ┆ 토의/토론 중심 ┆ 강의 중심 ┆ 동영상 등 ICT 활용 │
│ ---  ┆ ---    ┆ ---      ┆ ---            ┆ ---       ┆ ---                │
│ u32  ┆ u32    ┆ u32      ┆ u32            ┆ u32       ┆ u32                │
╞══════╪════════╪══════════╪════════════════╪═══════════╪════════════════════╡
│ 53   ┆ 63     ┆ 91       ┆ 15             ┆ 35        ┆ 62                 │
└──────┴────────┴──────────┴────────────────┴───────────┴────────────────────┘


In [61]:
# 
# 사. KIOST가 제공하는 연구윤리 교육 콘텐츠
# 
# 1) KIOST 연구윤리 교육 콘텐츠 확인 여부
# 

map = {
    1: "보지 못했다",
    2: "보았다"
}

video = df.group_by("Q14_1").len().rename({"Q14_1": "확인여부"}).with_columns(구분=pl.lit("연구윤리 동영상"))
newsletter = df.group_by("Q14_2").len().rename({"Q14_2": "확인여부"}).with_columns(구분=pl.lit("연구윤리 뉴스레터"))
qanda = df.group_by("Q14_3").len().rename({"Q14_3": "확인여부"}).with_columns(구분=pl.lit("연구윤리 Q&A"))
checklist = df.group_by("Q14_4").len().rename({"Q14_4": "확인여부"}).with_columns(구분=pl.lit("연구윤리 체크리스트"))


KIOST_contents = pl.DataFrame()
KIOST_contents = KIOST_contents.vstack(video).vstack(newsletter).vstack(qanda).vstack(checklist).with_columns(
    pl.col("확인여부").replace_strict(map),
)
KIOST_contents.write_csv("figure/3/사_1_KIOST_연구윤리_교육_콘텐츠_확인_여부.csv", include_bom=True)
KIOST_contents

확인여부,len,구분
str,u32,str
"""보았다""",110,"""연구윤리 동영상"""
"""보지 못했다""",57,"""연구윤리 동영상"""
"""보았다""",126,"""연구윤리 뉴스레터"""
"""보지 못했다""",41,"""연구윤리 뉴스레터"""
"""보지 못했다""",69,"""연구윤리 Q&A"""
"""보았다""",98,"""연구윤리 Q&A"""
"""보지 못했다""",85,"""연구윤리 체크리스트"""
"""보았다""",82,"""연구윤리 체크리스트"""


In [62]:
# bar = KIOST_contents.plot.bar().encode(
#     x=alt.X("확인여부:O", title=None, axis=None),
#     y=alt.Y("len:Q", title=None),
#     color=alt.Color("확인여부:N"),
#     # column=alt.Column("구분:N", title="KIOST 연구윤리 교육 콘텐츠 확인 여부", center=True, align="each"),
# ).properties(
#     width=120,
# )

# text = bar.mark_text(
#     align="center",
#     baseline="bottom",
# ).encode(
#     text="len",
# )

# layered_chart = alt.layer(bar, text)

# # Apply faceting to the layered chart
# final_chart = layered_chart.facet(
#     column=alt.Column("구분:N", title="KIOST 연구윤리 교육 콘텐츠 확인 여부")
# )

In [63]:
# 
# 사. KIOST가 제공하는 연구윤리 교육 콘텐츠
# 
# 2) KIOST 연구윤리 교육 콘텐츠 만족도
# 
#  Q15_1: 연구윤리 동영상 만족도
#  Q15_2: 연구윤리 뉴스레터 만족도
#  Q15_3: 연구윤리 Q&A 만족도
#  Q15_4: 연구윤리 체크리스트 만족도

map = {
    1: "매우 만족",
    2: "만족",
    3: "보통",
    4: "불만족",
    5: "매우 불만족"
}

print(chi2_test("Q15_1"))
basic_analysis("Q15_1")


{'성별': (4.08249206585958, 0.39495686513416556), '연령대': (15.021916170241457, 0.5230339343599888), '직위': (20.21493241677236, 0.4445588976847819), '최종학위': (6.615227233233345, 0.5786568466562789), '연구경력': (16.92331060674453, 0.15250495400568204), '연구분야': (18.993527951158917, 0.7523346794838984)}


C:\Users\hyeon\AppData\Local\Temp\ipykernel_26388\795108283.py:14: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)


(shape: (4, 2)
 ┌───────┬─────┐
 │ Q15_1 ┆ len │
 │ ---   ┆ --- │
 │ i64   ┆ u32 │
 ╞═══════╪═════╡
 │ 1     ┆ 18  │
 │ 2     ┆ 60  │
 │ 3     ┆ 33  │
 │ 4     ┆ 3   │
 └───────┴─────┘,
 shape: (4, 3)
 ┌───────┬─────┬──────┐
 │ Q15_1 ┆ len ┆ pct  │
 │ ---   ┆ --- ┆ ---  │
 │ i64   ┆ u32 ┆ f64  │
 ╞═══════╪═════╪══════╡
 │ 1     ┆ 18  ┆ 15.8 │
 │ 2     ┆ 60  ┆ 52.6 │
 │ 3     ┆ 33  ┆ 28.9 │
 │ 4     ┆ 3   ┆ 2.6  │
 └───────┴─────┴──────┘)

In [64]:
# 
# 사. KIOST가 제공하는 연구윤리 교육 콘텐츠
# 
# 2) KIOST 연구윤리 교육 콘텐츠 만족도
# 
#  Q15_1: 연구윤리 동영상 만족도
#  Q15_2: 연구윤리 뉴스레터 만족도
#  Q15_3: 연구윤리 Q&A 만족도
#  Q15_4: 연구윤리 체크리스트 만족도

map = {
    1: "매우 불만족",
    2: "불만족",
    3: "보통",
    4: "만족",
    5: "매우 만족"
}

print(chi2_test("Q15_2"))
basic_analysis("Q15_2")

{'성별': (2.6038238042981123, 0.6261459032738297), '연령대': (8.652731307441453, 0.9270229276475723), '직위': (18.348914208790074, 0.5644353614579621), '최종학위': (9.859615182256691, 0.27501772606551345), '연구경력': (7.048124677260015, 0.8544169385390981), '연구분야': (29.487749389678335, 0.20231326929961188)}


C:\Users\hyeon\AppData\Local\Temp\ipykernel_26388\795108283.py:14: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)


(shape: (4, 2)
 ┌───────┬─────┐
 │ Q15_2 ┆ len │
 │ ---   ┆ --- │
 │ i64   ┆ u32 │
 ╞═══════╪═════╡
 │ 1     ┆ 25  │
 │ 2     ┆ 70  │
 │ 3     ┆ 30  │
 │ 4     ┆ 5   │
 └───────┴─────┘,
 shape: (4, 3)
 ┌───────┬─────┬──────┐
 │ Q15_2 ┆ len ┆ pct  │
 │ ---   ┆ --- ┆ ---  │
 │ i64   ┆ u32 ┆ f64  │
 ╞═══════╪═════╪══════╡
 │ 1     ┆ 25  ┆ 19.2 │
 │ 2     ┆ 70  ┆ 53.8 │
 │ 3     ┆ 30  ┆ 23.1 │
 │ 4     ┆ 5   ┆ 3.8  │
 └───────┴─────┴──────┘)

In [65]:
# 
# 사. KIOST가 제공하는 연구윤리 교육 콘텐츠
# 
# 2) KIOST 연구윤리 교육 콘텐츠 만족도
# 
#  Q15_1: 연구윤리 동영상 만족도
#  Q15_2: 연구윤리 뉴스레터 만족도
#  Q15_3: 연구윤리 Q&A 만족도
#  Q15_4: 연구윤리 체크리스트 만족도

map = {
    1: "매우 불만족",
    2: "불만족",
    3: "보통",
    4: "만족",
    5: "매우 만족"
}

print(chi2_test("Q15_3"))
basic_analysis("Q15_3")

{'성별': (3.252437789369778, 0.5165032151401963), '연령대': (24.34468400164396, 0.08224186918149962), '직위': (26.887291240586606, 0.1384648578597767), '최종학위': (8.989279486808764, 0.3432011347722043), '연구경력': (31.556107980243187, 0.0016195269150460216), '연구분야': (21.119233628293735, 0.6316816311029427)}


C:\Users\hyeon\AppData\Local\Temp\ipykernel_26388\795108283.py:14: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)


(shape: (4, 2)
 ┌───────┬─────┐
 │ Q15_3 ┆ len │
 │ ---   ┆ --- │
 │ i64   ┆ u32 │
 ╞═══════╪═════╡
 │ 1     ┆ 25  │
 │ 2     ┆ 50  │
 │ 3     ┆ 29  │
 │ 4     ┆ 3   │
 └───────┴─────┘,
 shape: (4, 3)
 ┌───────┬─────┬──────┐
 │ Q15_3 ┆ len ┆ pct  │
 │ ---   ┆ --- ┆ ---  │
 │ i64   ┆ u32 ┆ f64  │
 ╞═══════╪═════╪══════╡
 │ 1     ┆ 25  ┆ 23.4 │
 │ 2     ┆ 50  ┆ 46.7 │
 │ 3     ┆ 29  ┆ 27.1 │
 │ 4     ┆ 3   ┆ 2.8  │
 └───────┴─────┴──────┘)

In [66]:
# 
# 사. KIOST가 제공하는 연구윤리 교육 콘텐츠
# 
# 2) KIOST 연구윤리 교육 콘텐츠 만족도
# 
#  Q15_1: 연구윤리 동영상 만족도
#  Q15_2: 연구윤리 뉴스레터 만족도
#  Q15_3: 연구윤리 Q&A 만족도
#  Q15_4: 연구윤리 체크리스트 만족도

map = {
    1: "매우 불만족",
    2: "불만족",
    3: "보통",
    4: "만족",
    5: "매우 만족"
}

print(chi2_test("Q15_4"))
basic_analysis("Q15_4")

{'성별': (3.6858029158702763, 0.45019417595621947), '연령대': (13.557008391804821, 0.6316781946951873), '직위': (15.066948896443797, 0.7725640432539091), '최종학위': (13.047458022607502, 0.11022712965458807), '연구경력': (18.422324888293332, 0.10345604245123408), '연구분야': (23.506073209311193, 0.4901121779456892)}


C:\Users\hyeon\AppData\Local\Temp\ipykernel_26388\795108283.py:14: DeprecationWarning: `aggregate_function='count'` input for `pivot` is deprecated. Please use `aggregate_function='len'`.
  target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)


(shape: (4, 2)
 ┌───────┬─────┐
 │ Q15_4 ┆ len │
 │ ---   ┆ --- │
 │ i64   ┆ u32 │
 ╞═══════╪═════╡
 │ 1     ┆ 20  │
 │ 2     ┆ 45  │
 │ 3     ┆ 28  │
 │ 4     ┆ 3   │
 └───────┴─────┘,
 shape: (4, 3)
 ┌───────┬─────┬──────┐
 │ Q15_4 ┆ len ┆ pct  │
 │ ---   ┆ --- ┆ ---  │
 │ i64   ┆ u32 ┆ f64  │
 ╞═══════╪═════╪══════╡
 │ 1     ┆ 20  ┆ 20.8 │
 │ 2     ┆ 45  ┆ 46.9 │
 │ 3     ┆ 28  ┆ 29.2 │
 │ 4     ┆ 3   ┆ 3.1  │
 └───────┴─────┴──────┘)

In [67]:
# 
# 사. KIOST가 제공하는 연구윤리 교육 콘텐츠
# 
# 3) KIOST 연구윤리 교육 콘텐츠별 수정 및 보완 요구 사항
# 
#  Q17: 연구윤리 동영상 만족도


print("연구윤리동영상", basic_analysis("Q17_1"))
print("연구윤리뉴스레터", basic_analysis("Q17_2"))
print("연구윤리Q&A", basic_analysis("Q17_3"))
print("연구윤리Checklist", basic_analysis("Q17_4"))
print("기타", basic_analysis("Q17_5"))

연구윤리동영상 (shape: (1, 2)
┌───────┬─────┐
│ Q17_1 ┆ len │
│ ---   ┆ --- │
│ i64   ┆ u32 │
╞═══════╪═════╡
│ 1     ┆ 60  │
└───────┴─────┘, shape: (1, 3)
┌───────┬─────┬───────┐
│ Q17_1 ┆ len ┆ pct   │
│ ---   ┆ --- ┆ ---   │
│ i64   ┆ u32 ┆ f64   │
╞═══════╪═════╪═══════╡
│ 1     ┆ 60  ┆ 100.0 │
└───────┴─────┴───────┘)
연구윤리뉴스레터 (shape: (1, 2)
┌───────┬─────┐
│ Q17_2 ┆ len │
│ ---   ┆ --- │
│ i64   ┆ u32 │
╞═══════╪═════╡
│ 2     ┆ 74  │
└───────┴─────┘, shape: (1, 3)
┌───────┬─────┬───────┐
│ Q17_2 ┆ len ┆ pct   │
│ ---   ┆ --- ┆ ---   │
│ i64   ┆ u32 ┆ f64   │
╞═══════╪═════╪═══════╡
│ 2     ┆ 74  ┆ 100.0 │
└───────┴─────┴───────┘)
연구윤리Q&A (shape: (1, 2)
┌───────┬─────┐
│ Q17_3 ┆ len │
│ ---   ┆ --- │
│ i64   ┆ u32 │
╞═══════╪═════╡
│ 3     ┆ 74  │
└───────┴─────┘, shape: (1, 3)
┌───────┬─────┬───────┐
│ Q17_3 ┆ len ┆ pct   │
│ ---   ┆ --- ┆ ---   │
│ i64   ┆ u32 ┆ f64   │
╞═══════╪═════╪═══════╡
│ 3     ┆ 74  ┆ 100.0 │
└───────┴─────┴───────┘)
연구윤리Checklist (shape: (1, 2)
┌───────┬────

In [68]:
# 
# 5) 연구윤리 Q&A 사례집에 대한 보완 사항
# 

basic_analysis("Q19")

(shape: (8, 2)
 ┌─────────────────────────────────┬─────┐
 │ Q19                             ┆ len │
 │ ---                             ┆ --- │
 │ str                             ┆ u32 │
 ╞═════════════════════════════════╪═════╡
 │                                 ┆ 1   │
 │ AI활용 관련 연구윤리 내용 추가  ┆ 2   │
 │ Q&A 제작 시 원내연구자의        ┆ 4   │
 │ 질의사항 반영                   ┆     │
 │ 교육 방법의 다양성              ┆ 3   │
 │ 구체적인 사례와 예시            ┆ 15  │
 │ 연구 윤리 위반 사례             ┆ 1   │
 │ 연구윤리 관련 교육과 정보제공   ┆ 5   │
 │ 홍보와 접근성 개선              ┆ 9   │
 └─────────────────────────────────┴─────┘,
 shape: (8, 3)
 ┌─────────────────────────────────┬─────┬──────┐
 │ Q19                             ┆ len ┆ pct  │
 │ ---                             ┆ --- ┆ ---  │
 │ str                             ┆ u32 ┆ f64  │
 ╞═════════════════════════════════╪═════╪══════╡
 │                                 ┆ 1   ┆ 2.5  │
 │ AI활용 관련 연구윤리 내용 추가  ┆ 2   ┆ 5.0  │
 │ Q&A 제작 시 원내연구자의        ┆ 4   ┆ 10.0 │
 │ 질의사항 반영        

In [69]:
# 
# 아. 연구윤리 컨설팅 희망 내용
# 

basic_analysis("Q20")

(shape: (17, 2)
 ┌─────────────────────────────────┬─────┐
 │ Q20                             ┆ len │
 │ ---                             ┆ --- │
 │ str                             ┆ u32 │
 ╞═════════════════════════════════╪═════╡
 │                                 ┆ 1   │
 │ .                               ┆ 1   │
 │ 구체적인 사례 중심의 연구윤리   ┆ 2   │
 │ 교육                            ┆     │
 │ 기타 고민 사항                  ┆ 5   │
 │ 논문 작성과 출판 관련 문제      ┆ 3   │
 │ …                               ┆ …   │
 │ 연구윤리 교육과 지침            ┆ 6   │
 │ 연국기관간 연구수행성과 및      ┆ 1   │
 │ 내용의 소유에 대한 내용         ┆     │
 │ 이해충돌                        ┆ 1   │
 │ 인공지능 및 AI 연구             ┆ 15  │
 │ 표절 및 유사도 검사             ┆ 5   │
 └─────────────────────────────────┴─────┘,
 shape: (17, 3)
 ┌─────────────────────────────────┬─────┬──────┐
 │ Q20                             ┆ len ┆ pct  │
 │ ---                             ┆ --- ┆ ---  │
 │ str                             ┆ u32 ┆ f64  │
 ╞══════════════════════════════

In [70]:
# 
# chi sqaure
# 

# from attributes import *
# from scipy import stats

# #
# # Chi-square test
# #
# personal_info = pl.read_csv("data/6_personal_info.csv")
# recognition_stats = df.join(personal_info, on="ID")

# def run_chi2_test(target_name):
#     chi_analysis = {}
#     for key, value in MAP_PERSON.items():
#         target = recognition_stats.pivot(target_name, index=value, values=target_name, aggregate_function='count', sort_columns=True).fill_null(0).drop(value)
#         t, p, dof, _ = stats.chi2_contingency(target.to_numpy())
#         chi_analysis[key] = {f"{key}_t":t, f"{key}_p":p}
#     return chi_analysis

# def run_chi(column, name):
#     dataframe = pl.DataFrame()
#     for c in column:
#         result = pl.from_dict(run_chi2_test(c)) \
#                     .unnest("sex") \
#                     .unnest("age") \
#                     .unnest("role") \
#                     .unnest("degree") \
#                     .unnest("experience") \
#                     .unnest("field")
#         dataframe = dataframe.vstack(result)
#     dataframe.write_csv(f"figure/4/{name}_chi.csv", include_bom=True)


# q6 = ["Q21_1", "Q21_2", "Q21_3", "Q21_4", "Q21_5"]
# q7 = ["Q22_1", "Q22_2", "Q22_3", "Q22_4", "Q22_5"]
# q8 = ["Q23"]
# q9 = ["Q24_1", "Q24_2", "Q24_3", "Q24_4", "Q24_5"]
# q10 = ["Q25_1", "Q25_2", "Q25_3", "Q25_4", "Q25_5", "Q25_6", "Q25_7", "Q25_8", "Q25_9", "Q25_10"]
# q11 = ["Q26"]
# q13 = ["Q28"]
# q14 = ["Q29"]
# q15 = ["Q29"]
# q16 = ["Q29"]
# # q17

# run_chi(q21, "가_연구윤리_규정_인지_검정")
# run_chi(q22, "나_연구윤리_규정_파악")
# run_chi(q23, "다_국가혁신개발법_인지")
# run_chi(q24, "라_국가혁신개발법_이해")
# run_chi(q25, "마_연구부정행위_인지")
# run_chi(q26, "바_연구부정행위_제보")
# run_chi(q27, "바_연구부정행위_검증")
# run_chi(q28, "사_연구부정행위_제보")
# run_chi(q29, "아_연구부정행위_검증")